In [3]:
pip show tensorflow

Name: tensorflow
Version: 2.4.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /root/anaconda3/envs/tensorflow37/lib/python3.7/site-packages
Requires: absl-py, wheel, tensorflow-estimator, flatbuffers, protobuf, numpy, six, termcolor, gast, wrapt, astunparse, grpcio, tensorboard, google-pasta, h5py, keras-preprocessing, typing-extensions, opt-einsum
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import tensorflow as tf

# 일반적인 선형회귀 과정

## Linear Model
- 1차 함수
- 선형성

## 선형모델에 대한 여러가지 표현

- 단순선형
$$
\begin{align*}
    f(x) &= b + ax \\
    f(x) &= w_0 + w_1x \\
    f(x) &= \beta_0 + \beta_1x \\
    f(x) &= X^{T}B
\end{align*}
$$

- 다중선형
$$
\begin{align*}
    f(x) &= \beta_0 \cdot 1 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n \\
    f(x) &= [1, x_1, x_2, ... , x_n] 
    \begin{bmatrix}
        \beta_0 \\
        \beta_1 \\
        \beta_2 \\
        ... \\
        \beta_n
    \end{bmatrix} \\
    f(x) &= X\cdot B \\
    f(x) &= [x_1, x_2, ... , x_n] 
    \begin{bmatrix}
        \beta_1 \\
        \beta_2 \\
        ... \\
        \beta_n
    \end{bmatrix} + b\\
    f(x) &= X\cdot B + b
\end{align*}
$$

## sample data

In [17]:
x = np.array([1,2,3,4], dtype=np.float64)
y = np.array([0, -1, -3, -3], dtype=np.float64)

## 손실함수

In [18]:
def mse(beta, x, y):
    yhat = beta[0] + beta[1] * x
    # yhat = np.dot(x, beta)
    return np.mean((y - yhat) ** 2)

## 기울기 함수

In [19]:
def grad_loss(beta, x, y):
    n = len(x)
    val = np.array([0., 0.])
    for i in range(n):
        error = beta[0] + beta[1] * x[i] - y[i]
        val += 2.0 * error * np.array([1.0, x[i]]) / n
    return val

## 최적화(gradient descent)

In [20]:
maxIter = 10
learning_rate = 0.01
beta0 = np.array([0., 0.])

for i in range(maxIter):
    grad = grad_loss(beta0, x, y)
    beta1 = beta0 - learning_rate * grad
    beta0 = beta1
    print(beta0, mse(beta0, x, y))

[-0.035 -0.115] 3.4255375
[-0.06355 -0.211  ] 2.5060663524999995
[-0.086729  -0.2911725] 1.8676128538753747
[-0.10543579 -0.35816017] 1.4241542579798327
[-0.12041907 -0.41416436] 1.116001185187483
[-0.13230247 -0.46101875] 0.9017364218329584
[-0.14160548 -0.50025082] 0.7526214237021915
[-0.14876083 -0.53313292] 0.6487151071519776
[-0.15412897 -0.56072494] 0.5761807603863539
[-0.15801014 -0.58390975] 0.5254173661714167


# 텐서플로우를 이용한 선형회귀 과정

## sample data

In [21]:
x = tf.constant([1,2,3,4], dtype=tf.float64)
y = tf.constant([0, -1, -3, -3], dtype=tf.float64)

In [22]:
a = tf.Variable([0], dtype=tf.float64) # 기울기, beta1, 변수의 영향력
b = tf.Variable([0], dtype=tf.float64) # y-절편, beta0, bias(편향), 상수항

## 손실함수, 최적화 함수

In [24]:
def linear_model(x):
    return b + a * x

def mse(yhat, y):
    return tf.reduce_mean(tf.pow(yhat - y, 2))

## 최적화 과정(학습)

In [26]:
maxIter = 10
optimizer = tf.optimizers.SGD(0.01)

for i in range(maxIter):
    
    # 기울기
    with tf.GradientTape() as grad:
        yhat = linear_model(x)
        loss = mse(yhat, y)
        
    # 기울기 계산
    gradients = grad.gradient(loss, [a, b])
    
    # a, b를 업데이트 
    optimizer.apply_gradients(zip(gradients, [a, b]))
    
    print('step: {}, loss: {}, a: {}, b: {}'.format(i, loss, a.numpy(), b.numpy()))

step: 0, loss: 3.425537526909821, a: [-0.211], b: [-0.06355]
step: 1, loss: 2.5060663898646043, a: [-0.29117249], b: [-0.086729]
step: 2, loss: 1.8676128927953963, a: [-0.35816017], b: [-0.10543579]
step: 3, loss: 1.4241542940276417, a: [-0.41416435], b: [-0.12041907]
step: 4, loss: 1.116001216503328, a: [-0.46101875], b: [-0.13230247]
step: 5, loss: 0.9017364479676788, a: [-0.50025081], b: [-0.14160548]
step: 6, loss: 0.7526214449278392, a: [-0.53313291], b: [-0.14876083]
step: 7, loss: 0.6487151240623148, a: [-0.56072493], b: [-0.15412897]
step: 8, loss: 0.5761807736743624, a: [-0.58390974], b: [-0.15801014]
step: 9, loss: 0.525417376512874, a: [-0.60342278], b: [-0.16065445]
